In [1]:
import time
import glob
import sys
import os
import uuid
import numpy as np
from PIL import Image, ImageDraw
import cv2
import pandas as pd
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [ ]:
import copy
import numpy as np
xml_path = "/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/p1/*.xml"
new_path = "/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/p1/"
def update_xml(xml_path):
    for i in sorted(glob.glob(xml_path)):
        name = i.split('.xml')[0]+'.jpg'
        tree2 = ET.parse(i)
        root2 = tree2.getroot()
        servlets2 = tree2.findall('object')
        flag=False
        for servlets in servlets2:
            if servlets[0].text=='table':
                flag=True
        if flag==True:
            #xml_path = new_path+i.split('/')[-1]
            xml_path = i
            img_path = i.split('.xml')[0]+'.jpg'
            
            check=False;f1=False;f2=False;f3=False;f4=False
            val=None
            image = cv2.imread(name)
            blank_image = np.zeros(image.shape, dtype=np.uint8)
            #cv2.fillPoly(mask, pts=[contours], color=(255,255,255))
            blank_image[:,0:image.shape[1]//2] = (255,255,255)      # (B, G, R)
            blank_image[:,image.shape[1]//2:image.shape[1]] = (255,255,255)
            m_xmin = 15000;m_ymin = 15000;m_xmax = -1;m_ymax = -1
            for servlets in servlets2:
#                 if servlets[0].text!='table':
                    
#                     xmin = int(servlets[4][0].text)
#                     ymin = int(servlets[4][1].text)
#                     xmax = int(servlets[4][2].text)
#                     ymax = int(servlets[4][3].text)
#                     root2.remove(servlets)
#                     ET.dump(root2)
#                     tree2.write(xml_path)
                if servlets[0].text=='table':
                    servlets[0].text = "cell"
#                     xmin = int(servlets[4][0].text)
#                     ymin = int(servlets[4][1].text)
#                     xmax = int(servlets[4][2].text)
#                     ymax = int(servlets[4][3].text)
#                     img = cv2.imread(img_path)
#                     crop_img = img[ymin:ymax,xmin:xmax,:]
#                     blank_image[ymin:ymax,xmin:xmax] = crop_img
#                     servlets[4][0].text = str(xmin-30)
#                     servlets[4][1].text= str(xmax-30)
#                     servlets[4][2].text = str(ymin-30)
#                     servlets[4][3].text = str(ymax-30)
                    ET.dump(root2)
                    tree2.write(xml_path)
                    
                              
            #cv2.imwrite(name,blank_image)
#             if val!=None:
#                 root2.remove(val)
#                 ET.dump(root2)
#                 tree2.write(xml_path)

update_xml(xml_path)

In [2]:
nb_dir = '/home/naresh/Tarento/primalaynet/layout-model-training/tools/table/'
#sys.path.append(nb_dir)
#sys.path.append(os.path.split(nb_dir)[0])

#import config
from src.services.get_table_cells import mask_tables 

import src.utilities.app_context as app_context
app_context.init()
app_context.application_context = dict({'task_id': 'BM-DUMMY'})





In [3]:
def draw_region(image_path,coords,save_dir,color=(255,125,0), thickness=3):
    
    image = cv2.imread(image_path)
    for region in coords :
            ground = region['boundingBox']['vertices']
            pts = []
            for pt in ground:
                pts.append([int(pt['x']) ,int(pt['y'])])
            cv2.polylines(image, [np.array(pts)],True, color, thickness)
#             cv2.putText(image, str(region['class']), (pts[0][0],pts[0][1]), font,  
#            2, (0,125,255), 3, cv2.LINE_AA)

    image_path = os.path.join(save_dir ,  '{}.png'.format(uuid.uuid4()))            
    cv2.imwrite(image_path , image)


In [6]:
df = pd.DataFrame(columns = ['ID', 'Label']);
ids =[]
label =[]
index =0
image_dir = '/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data'

image = glob.glob(image_dir + '/*.jpg')
for page_path in image:
    #path_to_text_file = save_dir+ page_path.split('/')[-1].split('.jpg')[0]+'.txt'
    #print(page_path)
#     if 'prima_double_column' not in page_path:
#         print(page_path)
    masked_image, reg  = mask_tables(page_path, check=False)
    #cv2.imwrite(image_path , image)
    #text_file = open(path_to_text_file, "w+")
    reg_str =''
    for idx,region in enumerate(reg):
        #cls = region['class'].lower()

        #if cls in ['footer','header','image','others','text','table']:
            #text_file.write(str(idx))
        reg_str = reg_str + str(region['boundingBox']['vertices'][0]['x'])+" "+str(region['boundingBox']['vertices'][0]['y'])+" "+str(region['boundingBox']['vertices'][2]['x'])+" "+str(region['boundingBox']['vertices'][2]['y'])+" "+'table'+' ' 
#                 text_file.write(str(cls_map[cls]))
#                 text_file.write(" "+str(region['boundingBox']['vertices'][0]['x']))
#                 text_file.write(" "+str(region['boundingBox']['vertices'][0]['y']))
#                 text_file.write(" "+str(region['boundingBox']['vertices'][2]['x']))
#                 text_file.write(" "+str(region['boundingBox']['vertices'][2]['y'])+"\n")
        #img = cv2.imread(page_path)
        #img2 = img[region['boundingBox']['vertices'][0]['y']:region['boundingBox']['vertices'][2]['y'],region['boundingBox']['vertices'][0]['x']:region['boundingBox']['vertices'][2]['x']]
        #print(img2)
        #cv2.imwrite(page_path.split('.jpg')[0]+str(idx)+'.jpg',img2)
    
    if len(reg_str)>0:
        print(page_path)
        #ids.append(page_path.split('/')[-1]+str(idx)+'.jpg')
        ids.append(page_path.split('/chunk_data/')[-1])
        label.append(reg_str)

        index = index+1


/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data/double_colourfont_100_0.5_0.5_1.5_300-converted-7.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data/double_colourfont_100_0.5_0.5_1.5_300-converted-2.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data/double_colourfont_100_0.5_0.5_1.5_300-converted-8.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data/double_colourfont_100_0.5_0.5_1.5_300-converted-6.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data/double_colourfont_100_0.5_0.5_1.5_300-converted-4.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data/double_colourfont_100_0.5_0.5_1.5_300-converted-3.jpg
/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data/double_colourfont_100_0.5_0.5_1.5_300-converte

In [7]:
df['ID']=ids
df['Label']=label

In [8]:
df.to_csv("/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/tmp_data/table.csv")

In [ ]:
df2 = df.dropna(axis=1)

In [30]:
df['ID'][0]

'/home/naresh/Tarento/primalaynet/layout-model-training/tools/generate_table_data/generated_data_labels/vis_w_3.5_0.2_0.5_200.jpg'

In [ ]:
save_dir = '/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_extraction_tool/samples'

In [ ]:
image_path = '/home/naresh/Tarento/primalaynet/layout-model-training/tools/Images/LAFEAS_LS-pdf-45_judgement_single_column_.jpg'

In [ ]:
masked_image, line_tables_regions  = mask_tables(image_path, check=False)

In [ ]:
draw_region(image_path,line_tables_regions,save_dir)

In [ ]:
import glob
xml_path = "/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/Annotations_updated/*.xml"
image_path = "/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/Annotations_updated/*.jpg"
text_file_path ="/home/naresh/Tarento/primalaynet/layout-model-training/tools/table_tagging/annpaths_list.txt"
text_file = open(text_file_path,'w+')
def rename_path(image_path,xml_path,by):
    for i,j in zip(sorted(glob.glob(image_path)),sorted(glob.glob(xml_path))):
        text_file.write(j+"\n")
        #os.rename(i, i.split('.jpg')[0]+'_'+by+'_'+'.jpg')
        #os.rename(j, j.split('.xml')[0]+'_'+by+'_'+'.xml')
rename_path(image_path,xml_path,'prima_double_column')